In [1]:
from model import LSTMModel
import torch
import sklearn.metrics as metrics

In [2]:
def get_metrics(model_name,model,dataloader):
    model.eval()
    total_correct=0
    total=0
    tp=[0]*4
    fp=[0]*4
    fn=[0]*4
    tn=[0]*4
    dict=[]
    for i in range(4):
        dict.append([0]*4)
    for i,(x,y) in enumerate(dataloader):
        y_pred=model(x)
        y_pred=torch.argmax(y_pred,dim=1)
        y=torch.argmax(y,dim=1)
        total_correct+=torch.sum(y_pred==y).item()
        total+=len(y)
        for i in range(4):
            tp[i]+=torch.sum((y_pred==i)&(y==i)).item()
            fp[i]+=torch.sum((y_pred==i)&(y!=i)).item()
            fn[i]+=torch.sum((y_pred!=i)&(y==i)).item()
            tn[i]+=torch.sum((y_pred!=i)&(y!=i)).item()
        for j in range(len(y)):
            dict[y_pred[j]][y[j]]+=1
    acc=total_correct/total
    accuracy_per_class=[0]*4
    precision=[0]*4
    recall=[0]*4
    f1=[0]*4
    for i in range(4):
        accuracy_per_class[i]=tp[i]/(tp[i]+fn[i])
        if tp[i]+fp[i]!=0:
            precision[i]=tp[i]/(tp[i]+fp[i])
        if tp[i]+fn[i]!=0:
            recall[i]=tp[i]/(tp[i]+fn[i])
        if precision[i]+recall[i]!=0:
            f1[i]=2*precision[i]*recall[i]/(precision[i]+recall[i])

    print('Accuracy:',acc)
    fpr=[fp[i]/(fp[i]+tn[i]) for i in range(4)]
    fnr=[fn[i]/(fn[i]+tp[i]) for i in range(4)]
    print('Classes:',['negative','learning','recall','recognition_familiar'])
    for i in range(4):
        print(accuracy_per_class[i],f1[i],recall[i],precision[i],fpr[i],fp[i],fnr[i],fn[i])
    print('Confusion matrix:')
    print(dict)




In [17]:
from sklearn import metrics
sf=torch.nn.Softmax()
def get_aoc(model_name,model,dataloader):
    ally_pred=[]
    ally=[]
    model.eval()
    l=[]
    for i,(x,y) in enumerate(dataloader):
        y=torch.argmax(y,dim=1)
        y=torch.nn.functional.one_hot(y,num_classes=4)
        y_pred=model(x)
        y_pred=sf(y_pred)
        for i in range(len(y)):
            ally.append(y[i])
            ally_pred.append(y_pred[i].detach().numpy())
    print(len(ally),len(ally_pred))
    print(len(ally[0]),len(ally_pred[0]))
    auc=metrics.roc_auc_score(ally,ally_pred,multi_class='ovr')
    l.append(auc)
    return l

    

In [18]:
for x in ['model2.5/2_16/model53.pt','model2.5/2_32/model23.pt','model2.5/2_64/model30.pt','model2.5/3_16/model63.pt','model2.5/3_32/model29.pt','model2.5/3_64/model26.pt','model2.5/4_10/model29.pt','model2.5/4_16/model51.pt','model2.5/4_32/model48.pt','model2.5/4_64/model64.pt']:
    print(x)
    second=x.split('/')[0]
    second=second[5:]
    val_dataloader=torch.load('val'+second+'_dataloader.pt')
    test_dataloader=torch.load('test'+second+'_dataloader.pt')
    model=torch.load(x)
    l1= get_aoc(x,model,val_dataloader)
    l2= get_aoc(x,model,test_dataloader)
    for i in range(4):
        print(l1[i])
        print(l2[i])
        print('\n')

model2.5/2_16/model53.pt
2585 2585
4 4


/Users/saksham/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/multiclass.py:150: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  y = np.asarray(y)
/Users/saksham/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/multiclass.py:154: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to eithe

AttributeError: 'bool' object has no attribute 'any'

In [ ]:
for x in ['model3/2_16/model45.pt','model3/2_32/model20.pt','model3/2_64/model25.pt','model3/3_16/model39.pt','model3/3_32/model29.pt','model3/3_64/model26.pt','model3/4_10/model30.pt','model3/4_16/model45.pt','model3/4_32/model37.pt','model3/4_64/model66.pt']:
    print(x)
    second=x.split('/')[0]
    second=second[5:]
    val_dataloader=torch.load('val'+second+'_dataloader.pt')
    test_dataloader=torch.load('test'+second+'_dataloader.pt')
    model=torch.load(x)
    l1= get_aoc(x,model,val_dataloader)
    l2= get_aoc(x,model,test_dataloader)
    for i in range(4):
        print(l1[i])
        print(l2[i])
        print('\n')

In [ ]:
for x in ['model3.5/2_16/model43.pt','model3.5/2_32/model39.pt','model3.5/2_64/model31.pt','model3.5/3_16/model27.pt','model3.5/3_32/model29.pt','model3.5/3_64/model29.pt','model3.5/4_10/model56.pt','model3.5/4_16/model50.pt','model3.5/4_32/model23.pt','model3.5/4_64/model68.pt']:
    print(x)
    second=x.split('/')[0]
    second=second[5:]
    val_dataloader=torch.load('val'+second+'_dataloader.pt')
    test_dataloader=torch.load('test'+second+'_dataloader.pt')
    model=torch.load(x)
    l1= get_aoc(x,model,val_dataloader)
    l2= get_aoc(x,model,test_dataloader)
    for i in range(4):
        print(l1[i])
        print(l2[i])
        print('\n')